# Import libraries


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [ ]:
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=46b85f8e1316598962d615e22559b4cd52dea62c6d22a23f350512d844dc9260
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [ ]:
import pandas as pd
# from datasets import Dataset

from google.colab import drive
drive.mount("/content/gdrive")

import datasets
import random
random.seed(10)

Mounted at /content/gdrive


# Read dataset

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/train_sport.csv', sep=';')
validation = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/validation_sport.csv', sep=';')
test = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/test_sport.csv', sep=';')

In [ ]:
train.head()

,document,summary,id,URL,category
0,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548,http://web.archive.org/web/20160403040918/http...,formula1
1,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833,http://web.archive.org/web/20151017072732/http...,rugby-union
2,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467,http://web.archive.org/web/20160404024929/http...,wales
3,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845,http://web.archive.org/web/20170729030028/http...,football
4,"The Bath-born player, 28, has made 36 appearan...",Newport Gwent Dragons number eight Ed Jackson ...,38900884,http://web.archive.org/web/20170208052041/http...,rugby-union


In [ ]:
train['text']=train['document'].map(lambda x: x[:1000])
validation['text']=validation['document'].map(lambda x: x[:1000])
test['text']=test['document'].map(lambda x: x[:1000])

In [ ]:
dd = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(train[["text", "summary"]]),
    "validation": datasets.Dataset.from_pandas(validation[["text", "summary"]]),
    "test": datasets.Dataset.from_pandas(test[["text", "summary"]])
})

In [ ]:
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 49802
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 2784
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 2830
    })
})

# Definition pipeline

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dd.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/pegasus-xsum")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

# Fine tuning of the model

In [ ]:
tokenized_dataset["train"]=tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
tokenized_dataset["validation"]=tokenized_dataset["validation"].shuffle(seed=42).select(range(1000))

In [ ]:
import torch

training_args = Seq2SeqTrainingArguments(
    output_dir="sport_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=4,
    predict_with_generate=True,
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

temp_file = '/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Model/pegasus_fine_tuning.pth'

#The second saves and loads the entire model:
torch.save(model, temp_file)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: summary, text. If summary, text are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10000
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
  Number of trainable parameters = 568699904


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.119900,1.374207,0.492700,0.271900,0.413700,0.413900,23.233000
2,1.068200,1.379940,0.493300,0.272500,0.413300,0.413400,23.344000
3,1.034500,1.390492,0.492300,0.268000,0.412100,0.412200,23.467000
4,1.004200,1.393950,0.492000,0.267500,0.410900,0.411100,23.473000


Output streaming troncato alle ultime 5000 righe.
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_lengt

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="sport_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=4,
    predict_with_generate=True,
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

temp_file = '/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Model/pegasus_fine_tuning2.pth'

#The second saves and loads the entire model:
torch.save(model, temp_file)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: summary, text. If summary, text are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.908100,1.446854,0.490500,0.268000,0.410600,0.411000,23.658000
2,0.907800,1.449163,0.491800,0.267500,0.410500,0.410400,23.679000
3,0.903600,1.452267,0.490100,0.264300,0.407700,0.408100,23.976000
4,0.924100,1.446594,0.492200,0.268300,0.411100,0.411300,23.872000


Output streaming troncato alle ultime 5000 righe.
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_lengt

# Evaluation on test set

In [ ]:
trainer.evaluate(tokenized_dataset["test"])

The following columns in the evaluation set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: summary, text. If summary, text are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2830
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



Output streaming troncato alle ultime 5000 righe.
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config Generation

{'eval_loss': 1.4132386445999146,
 'eval_rouge1': 0.4969,
 'eval_rouge2': 0.2747,
 'eval_rougeL': 0.4181,
 'eval_rougeLsum': 0.4181,
 'eval_gen_len': 23.6131,
 'eval_runtime': 833.4948,
 'eval_samples_per_second': 3.395,
 'eval_steps_per_second': 0.849,
 'epoch': 4.0}

Il modello ottenuto tramite fine tuning migliora i valori delle statistiche Rouge, in particolare si osserva un valore rilevante per la statistica Rouge-1 vicino al valore di 0.50.

## First observation

In [ ]:
out1 = trainer.predict(tokenized_dataset['test'])

The following columns in the test set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



In [ ]:
generated_summaries1 = []
generated_summaries1.append(tokenizer.decode(out1[0][0], skip_special_tokens=True))

In [ ]:
generated_summaries1

['West Bromwich Albion have appointed Reading director of football Nick Hammond as their new head coach.']

In [ ]:
test['summary'].iloc[0]

'West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.'

## Second observation

In [ ]:
out2 = trainer.predict(tokenized_dataset['train'])


The following columns in the test set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.6,
  "max_length": 64,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



In [ ]:
generated_summaries2 = []
generated_summaries2.append(tokenizer.decode(out2[0][1], skip_special_tokens=True))

In [ ]:
generated_summaries2

['Glasgow began their Pro12 campaign with a bonus-point win over Newport Gwent Dragons at Scotstoun.']

In [ ]:
train['summary'].iloc[1]

'Defending Pro12 champions Glasgow Warriors bagged a late bonus-point victory over the Dragons despite a host of absentees and two yellow cards.'